In [1]:
! pip install -U transformers datasets evaluate
! pip install -U accelerate
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from transformers import AutoTokenizer, EarlyStoppingCallback, IntervalStrategy
import torch
from torch.utils.data import DataLoader, TensorDataset

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 64.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 45.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.5 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 5.6 MB/s eta 0:00:00


In [2]:
df = pd.read_csv(r'soc_202311261432.csv')
len(df)

59121

In [3]:
df = df.fillna('N/A')
df.head()

,CASE_ID,SOC_CODE,SOC_TITLE,JOB_DUTIES
0,21291992,53-3032.00,Heavy and Tractor-Trailer Truck Drivers,Drive a tractor-trailer combination or a truck...
1,21293592,37-3013.00,Tree Trimmers and Pruners,Trims and/or removes trees by climbing with a...
2,21291654,37-3011.00,Landscaping and Groundskeeping Workers,Landscape or maintain grounds using hand or po...
3,21286051,49-9045.00,"Refractory Materials Repairers, Except Brickma...",Reline or repair interior of refractory vessel...
4,21295310,37-3011.00,Landscaping and Groundskeeping Workers,Landscape or maintain properties using tools o...


In [4]:
label2id = {label: idx for idx, label in enumerate(df['SOC_CODE'].unique())}
id2label = {v: k for k, v in label2id.items()}

In [5]:
df['SOC_CODE'] = df['SOC_CODE'].apply(lambda x: label2id[x])

In [6]:
X_train, X_test, y_train, y_test = train_test_split(df['JOB_DUTIES'], df['SOC_CODE'], test_size=0.2, random_state=42)

In [7]:
df_train = pd.DataFrame({'text':X_train, 'label':y_train})
df_test = pd.DataFrame({'text':X_test, 'label':y_test})

In [8]:
from datasets import DatasetDict, Dataset

# Convert dataframe to dataset
ds_train = Dataset.from_pandas(df_train)
ds_test = Dataset.from_pandas(df_test)

ds_train[0]

{'text': 'Tasks include: Operate vehicles and powered equipment, such as mowers, tractors, twin-axle vehicles, snow blowers, chainsaws, electric clippers, sod cutters, and pruning saws. Mow or edge lawns. Shovel snow and salt walks, driveways, or parking lots. Care for established lawns by mulching, aerating, weeding, grubbing, removing thatch, or trimming or edging around flower beds, walks, or walls.  Use hand tools, such as shovels, rakes, pruning saws, hedge or brush trimmers, or axes. Prune or trim trees, shrubs, or hedges. Gather and remove litter. Maintain or repair tools, equipment, or structures, such as building, greenhouses, fences, or benches, using hand or power tools. Mix and spray fertilizers, herbicides, or insecticides onto grass, shrubs, or trees.',
 'label': 2,
 '__index_level_0__': 27450}

In [9]:
from transformers import AutoTokenizer

# Load pre-trained BERT tokenizer and model
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [10]:
tokenizer

DistilBertTokenizerFast(name_or_path='distilbert-base-uncased', vocab_size=30522, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [11]:
def tokenize_dataset(data):
    return tokenizer(data['text'], truncation=True, padding=True)

In [12]:
# Tokenize input data
tokenized_ds_train = ds_train.map(tokenize_dataset, batched=True)
tokenized_ds_test = ds_test.map(tokenize_dataset, batched=True)

Map:   0%|          | 0/47296 [00:00<?, ? examples/s]

Map:   0%|          | 0/11825 [00:00<?, ? examples/s]

In [13]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [14]:
# ! pip install -U tensorflow

In [15]:
import evaluate

accuracy = evaluate.load("accuracy")

In [16]:
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [17]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained(
    'distilbert-base-uncased', num_labels=len(label2id), id2label=id2label, label2id=label2id
)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'pre_classifier.weight', 'classifier.weight', 'pre_classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [18]:
import accelerate
import transformers

transformers.__version__, accelerate.__version__

('4.36.0', '0.25.0')

In [19]:
tokenized_ds_train.features

{'text': Value(dtype='string', id=None),
 'label': Value(dtype='int64', id=None),
 '__index_level_0__': Value(dtype='int64', id=None),
 'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None),
 'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None)}

In [20]:
training_args = TrainingArguments(
    output_dir='SOC_classification_model',
    learning_rate=2e-5,
    seed=42,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=5,
    weight_decay=0.01,
    save_strategy='epoch',
    # save_steps=100, change above to Interval.steps
    evaluation_strategy='epoch',
    # eval_steps=100,
    load_best_model_at_end=True,
    push_to_hub=False
)

In [21]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_ds_train,
    eval_dataset=tokenized_ds_test,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)

In [22]:
trainer.train()

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy
1,0.741200,0.749545,0.841860
2,0.572600,0.625792,0.869429


Epoch,Training Loss,Validation Loss,Accuracy
1,0.741200,0.749545,0.841860
2,0.572600,0.625792,0.869429
3,0.409300,0.588417,0.881099
4,0.366700,0.592213,0.887019
5,0.316100,0.587696,0.888372


TrainOutput(global_step=29560, training_loss=0.5928866550306183, metrics={'train_runtime': 12449.7672, 'train_samples_per_second': 18.995, 'train_steps_per_second': 2.374, 'total_flos': 3.16152725766144e+16, 'train_loss': 0.5928866550306183, 'epoch': 5.0})

In [ ]:
model.save_pretrained('model')
tokenizer.save_pretrained('tokenizer')

('tokenizer/tokenizer_config.json',
 'tokenizer/special_tokens_map.json',
 'tokenizer/vocab.txt',
 'tokenizer/added_tokens.json',
 'tokenizer/tokenizer.json')

In [ ]:
from transformers import pipeline

text = "Drive a tractor-trailer combination or a truck with a capacity of at least 26,001 pounds. Secure cargo for transport, using ropes, blocks, chain, binders, or covers. Maneuver trucks into loading or unloading positions, following signals from loading crew and checking that vehicle and loading equipment are properly positioned. Perform basic vehicle maintenance tasks, such as adding oil, fuel, or radiator fluid or performing minor repairs. Load or unload trucks or help others with loading or unloading, using special loading-related equipment or other equipment as necessary. Remove debris from loaded trailers."

classifier = pipeline(task="text-classification", model="model", tokenizer='tokenizer')
classifier(text)

[{'label': '53-3032.00', 'score': 0.9769483208656311}]

In [ ]:
!pip install pickle5

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.1/132.1 kB 1.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pickle5: filename=pickle5-0.0.11-cp310-cp310-linux_x86_64.whl size=255314 sha256=2c8a11ccddab207f9d890ef02c7d6948a5cd0b6d56208dc59106503703705334
  Stored in directory: /root/.cache/pip/wheels/7d/14/ef/4aab19d27fa8e58772be5c71c16add0426acf9e1f64353235c
Successfully built pickle5


In [ ]:
import pickle

pickle.dump(model, open('model.sav', 'wb'))

In [ ]:
pickle.dump(tokenizer, open('tokenizer.sav', 'wb'))


In [ ]:
loaded_model = pickle.load(open('model.sav', 'rb'))
loaded_tokenizer = pickle.load(open('tokenizer.sav', 'rb'))

In [ ]:
text = "Drive a tractor-trailer combination or a truck with a capacity of at least 26,001 pounds. Secure cargo for transport, using ropes, blocks, chain, binders, or covers. Maneuver trucks into loading or unloading positions, following signals from loading crew and checking that vehicle and loading equipment are properly positioned. Perform basic vehicle maintenance tasks, such as adding oil, fuel, or radiator fluid or performing minor repairs. Load or unload trucks or help others with loading or unloading, using special loading-related equipment or other equipment as necessary. Remove debris from loaded trailers."

classifier = pipeline(task="text-classification", model=loaded_model, tokenizer=loaded_tokenizer, device="cpu")
classifier(text)

[{'label': '53-3032.00', 'score': 0.9769483208656311}]

In [ ]:
torch.save(model, open('torch_model.sav', 'wb'))
torch.save(tokenizer, open('torch_tokenizer.sav', 'wb'))

In [ ]:
!pip install fastapi
!pip install uvicorn
!pip install pydantic
!pip install requests
!pip install pypi-json
!pip install pyngrok
!pip install nest-asyncio